#Q-learning 실습
시작점에서 탐욕 정책으로 이동하며 경로 기록
-> 터미널 도착 시 보상을 확정
-> 그 보상을 점차 줄어가며(당장의 확실하고 가까운 보상을 중요하게 여기고, 미래의 불확실한 보상을 덜 중요하게 여김) 경로를 거꾸로 따라 Q값을 갱신
-> 초기화 후 다음 에피소드 반복

In [2]:
import numpy as np

BOARD_ROWS = 3
BOARD_COLS = 4
WIN_STATE = (0, 3) #에이전트가 도달하면 +1 보상
LOSE_STATE = (1, 3) #에이전트가 도달하면 -1 보상
BLOCKED_STATE = (1, 1) #장애물 존재
START = (2, 0) #에피소드가 시작되는 위치
DETERMINISTIC = False #False이면 확률적으로 미끄러짐이 발생
class State: #환경
    def __init__(self, state = START): #에이전트의 현재 위치를 저장
        self.state = state
        self.isEnd = False #현재 상태 win/lose 여부
        self.determine = DETERMINISTIC

    def giveReward(self): #보상함수
        if self.state == WIN_STATE:
            return 1
        elif self.state == LOSE_STATE:
            return -1
        else:
            return 0

    def isEndFunc(self): #종료 상태 판별
        if (self.state == WIN_STATE) or (self.state == LOSE_STATE):
            self.isEnd = True

    def _chooseActionProb(self, action): #확률적 행동 처리(미끄러짐)
    #불확실성을 구현
        if action == "U": #80% 위쪽, 10% 왼쪽, 10% 오른쪽
            return np.random.choice(["U", "L", "R"], p = [0.8, 0.1, 0.1])
        if action == "D":
            return np.random.choice(["D", "L", "R"], p = [0.8, 0.1, 0.1])
        if action == "L":
            return np.random.choice(["L", "U", "D"], p = [0.8, 0.1, 0.1])
        if action == "R":
            return np.random.choice(["R", "U", "D"], p = [0.8, 0.1, 0.1])

    def nxtPosition(self, action): #다음 위치 계산
        if self.determine: #deterministic 모드일 때 해당 방향으로 이동
            if action == "U":
                nxtState = (self.state[0] - 1, self.state[1])
            elif action == "D":
                nxtState = (self.state[0] + 1, self.state[1])
            elif action == "L":
                nxtState = (self.state[0], self.state[1] - 1)
            else:
                nxtState = (self.state[0], self.state[1] + 1)
            self.determine = False
        else: #stochastic 모드일 때 방향이 확률적으로 변할 수 있음
            action = self._chooseActionProb(action)
            self.determine = True
            nxtState = self.nxtPosition(action)

        if (nxtState[0] >= 0) and (nxtState[0] <= 2): #경계 조건 확인
            if (nxtState[1] >= 0) and (nxtState[1] <= 3): #보드 안에 있는지
                if nxtState != BLOCKED_STATE: #장애물인지
                    return nxtState #유효하지 않으면 제자리 유지
        return self.state

class Agent: #에이전트(Q-learning 학습자)

    def __init__(self):
        self.states = []
        self.actions = ["U", "D", "L", "R"]
        self.State = State()
        self.isEnd = self.State.isEnd
        self.lr = 0.2 #학습률
        self.decay_gamma = 0.9 #감소율

        #Q-Table 초기화
        self.Q_values = {}
        for i in range(BOARD_ROWS):
            for j in range(BOARD_COLS):
                self.Q_values[(i, j)] = {}
                for a in self.actions:
                    self.Q_values[(i, j)][a] = 0 #각 행동의 초기Q값을 0으로 세팅

    def chooseAction(self): #행동 선택
        max_nxt_reward = 0
        action = ""

        for a in self.actions:
            current_position = self.State.state
            nxt_reward = self.Q_values[current_position][a]
            if nxt_reward >= max_nxt_reward:
                action = a
                max_nxt_reward = nxt_reward
                #현재 상태에서 Q값이 가장 큰 행동을 선택
        return action

    def takeAction(self, action): #행동 수행 후 상태 이동
        position = self.State.nxtPosition(action)
        return State(state = position)
        #선택한 행동을 환경에 전달 -> 새로운 state 반환

    def reset(self): #에피소드 종료 후 초기화
        self.states = []
        self.State = State()
        self.isEnd = self.State.isEnd

    def play(self, episodes = 10): #학습 과정
        i = 0
        while i < episodes:
            if self.State.isEnd:
                reward = self.State.giveReward()
                for a in self.actions:
                    self.Q_values[self.State.state][a] = reward
                    #현재 상태가 종료 상태하면 -> 보상을 해당 상태의 Q값에 반영

                for s in reversed(self.states): #Q값 역전파 업데이트
                    current_q_value = self.Q_values[s[0]][s[1]]
                    reward = current_q_value + self.lr * (self.decay_gamma * reward - current_q_value)
                    self.Q_values[s[0]][s[1]] = round(reward, 3)
                    #s[0]은 state, s[1]은 action
                self.reset()
                i += 1 #다음 에피소드로 이동
            else: #에피소드 진행 중
                action = self.chooseAction() #종료가 아니라면 행동 선택
                self.states.append([(self.State.state), action])
                self.State = self.takeAction(action)
                self.State.isEndFunc()
                self.isEnd = self.State.isEnd

ag = Agent()

ag.play(1000) #에피소드 1000번 학습
print("latest Q-values ... \n")
print(ag.Q_values) #최종 Q-Table 출

latest Q-values ... 

{(0, 0): {'U': 0, 'D': 0, 'L': 0, 'R': 0.682}, (0, 1): {'U': 0, 'D': 0, 'L': 0, 'R': 0.787}, (0, 2): {'U': 0, 'D': 0, 'L': 0, 'R': 0.894}, (0, 3): {'U': 1, 'D': 1, 'L': 1, 'R': 1}, (1, 0): {'U': 0, 'D': 0, 'L': 0, 'R': 0.326}, (1, 1): {'U': 0, 'D': 0, 'L': 0, 'R': 0}, (1, 2): {'U': 0, 'D': 0, 'L': 0.356, 'R': -0.18}, (1, 3): {'U': -1, 'D': -1, 'L': -1, 'R': -1}, (2, 0): {'U': 0, 'D': 0, 'L': 0.071, 'R': -0.001}, (2, 1): {'U': 0, 'D': 0, 'L': 0.093, 'R': -0.005}, (2, 2): {'U': 0, 'D': 0, 'L': 0.112, 'R': -0.027}, (2, 3): {'U': 0, 'D': 0, 'L': 0, 'R': -0.157}}
